In [1]:
#Import basic packages
import os
import numpy as np
import pandas as pd
import csv
import regex as re
import datetime as dt 

In [2]:
org_ori_df = pd.read_csv(R"d:/msc-project/data/pre-processed/organisations_preprocessed.csv",encoding='utf-8',
                         index_col='org_uuid',
                         parse_dates= ['founded_on','went_public_on','acquired_on','closed_on','last_funding_on'])

In [3]:
org_ori_df.columns

Index(['name', 'legal_name', 'homepage_url', 'country_code', 'state_code',
       'region', 'city', 'address', 'postal_code', 'status',
       'short_description', 'category_list', 'category_groups_list',
       'num_funding_rounds', 'total_funding_usd', 'total_funding',
       'total_funding_currency_code', 'founded_on', 'last_funding_on',
       'closed_on', 'employee_count', 'email', 'phone', 'facebook_url',
       'linkedin_url', 'twitter_url', 'logo_url', 'primary_role', 'num_exits',
       'description', 'ipo_uuid', 'stock_exchange_symbol', 'stock_symbol',
       'went_public_on', 'share_price_usd', 'share_price',
       'share_price_currency_code', 'valuation_price_usd', 'valuation_price',
       'valuation_price_currency_code', 'money_raised_usd', 'money_raised',
       'money_raised_currency_code', 'acquisition_uuid', 'acquirer_uuid',
       'acquirer_name', 'acquirer_country_code', 'acquirer_state_code',
       'acquirer_region', 'acquirer_city', 'acquisition_type', 'acquired

In [4]:
funding_rounds_df = pd.read_csv(R"d:/msc-project/data/pre-processed/funding_rounds_preprocessed.csv",encoding='utf-8', 
            index_col='funding_round_uuid',
           parse_dates=['announced_on'])

In [5]:
funding_rounds_df.columns

Index(['name', 'country_code', 'state_code', 'region', 'city',
       'investment_type', 'announced_on', 'raised_amount_usd', 'raised_amount',
       'raised_amount_currency_code', 'post_money_valuation_usd',
       'post_money_valuation', 'post_money_valuation_currency_code',
       'investor_count', 'org_uuid', 'org_name', 'lead_investor_uuids'],
      dtype='object')

In [6]:
len(funding_rounds_df)

364336

In [7]:
funding_rounds_df.investment_type.value_counts()

seed                     102870
series_unknown            61708
series_a                  42105
series_b                  21084
grant                     20373
angel                     18847
pre_seed                  17224
debt_financing            14249
private_equity            11158
series_c                   9801
convertible_note           7386
post_ipo_equity            6416
equity_crowdfunding        6315
non_equity_assistance      5381
undisclosed                5296
series_d                   4126
corporate_round            3335
series_e                   1715
post_ipo_debt              1420
product_crowdfunding       1025
secondary_market            784
initial_coin_offering       688
series_f                    611
series_g                    210
series_h                     89
post_ipo_secondary           89
series_i                     19
series_j                     12
Name: investment_type, dtype: int64

In [8]:
#create processed organisation
org_processing_df = org_ori_df[['name','status','founded_on','last_funding_on']]

In [9]:
#create groupbyobject for investment_type
investment_type = funding_rounds_df.groupby('investment_type')

In [10]:
org_processing_df.head(3)

,name,status,founded_on,last_funding_on
org_uuid,,,,
e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,acquired,2005-06-01,2008-05-19
5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,acquired,2004-10-11,2016-09-13
df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,ipo,2004-02-04,2014-10-17


In [11]:
#create df of company with seed funding
seed_company_df = investment_type.get_group('seed')[['org_uuid','announced_on','lead_investor_uuids']]
seed_company_df.dropna(subset=['announced_on'],inplace=True)
seed_company_df.rename(columns={'announced_on':'seed_date','lead_investor_uuids':'lead_investor_uuids_seed'},inplace=True)
len(seed_company_df)

102870

In [12]:
#drop duplicate of companies with more than one seed date. keep only the first seed date.
seed_company_df.sort_values(by=['seed_date'],inplace=True)
seed_company_df_v1 = seed_company_df.drop_duplicates(subset=['org_uuid'],keep='first')

In [13]:
len(seed_company_df_v1)

78970

In [14]:
seed_company_df_v1.head(3)

,org_uuid,seed_date,lead_investor_uuids_seed
funding_round_uuid,,,
29c62cb5-bdd3-407a-a60f-a235fe819e90,65d0628c-d1d8-4699-9e61-6ef3200839ab,1950-09-20,NaN
24093d19-1f1d-ae85-64fe-557d2d08b368,1e4f199c-363b-451b-a164-f94571075ee5,1968-07-31,NaN
60fa2775-2c45-44d2-a649-bba7bf89338e,2e2cbc7a-bb1a-bd6d-3c6d-92a6def15d0a,1969-01-01,NaN


In [15]:
#create df of company with series_a funding
series_a_company_df = investment_type.get_group('series_a')[['org_uuid','announced_on','lead_investor_uuids']]
series_a_company_df.dropna(subset=['announced_on'],inplace=True)
series_a_company_df.rename(columns={'announced_on':'series_a_date','lead_investor_uuids':'lead_investor_uuids_series_a'},inplace=True)
len(series_a_company_df)

42105

In [16]:
#drop companies with more than one series a funding. keep only the first series a date
series_a_company_df.sort_values(by=['series_a_date'],inplace=True)
series_a_company_df_v1 = series_a_company_df.drop_duplicates(subset=['org_uuid'],keep='first')
len(series_a_company_df_v1)

37375

In [17]:
series_a_company_df_v1.head(3)

,org_uuid,series_a_date,lead_investor_uuids_series_a
funding_round_uuid,,,
9184700e-351d-4677-b240-ad2cc3b59d43,79ff0cd7-b392-41e5-80df-870bb2b40d96,1980-09-04,NaN
dba36c7a-4142-c372-d426-7784a6de74ee,27bb6fe7-0a5d-01c6-30af-55bd8e94c8e6,1981-09-01,NaN
46c353a8-2491-70cc-4b6a-b89a522fefdc,322eee3f-a036-651f-5754-0f8759374699,1982-06-01,NaN


In [18]:
#create df of company with series_uknown funding. Possibility of series to be series a
series_unknown_company_df = investment_type.get_group('series_unknown')[['org_uuid','announced_on']]
series_unknown_company_df.dropna(subset=['announced_on'],inplace=True)
series_unknown_company_df.rename(columns={'announced_on':'series_unknown_date'},inplace=True)
len(series_unknown_company_df)

61708

In [19]:
#drop companies with more than one series_unknown funding. keep only the first funding
series_unknown_company_df.sort_values(by=['series_unknown_date'],inplace=True)
series_unknown_company_df_v1 = series_unknown_company_df.drop_duplicates(subset=['org_uuid'],keep='first')
len(series_unknown_company_df_v1)

45102

In [20]:
#merge main file with seed information
org_processing_df_v1 = pd.merge(org_processing_df,
                                seed_company_df,
                                on='org_uuid',
                                how='left')

In [21]:
len(org_processing_df_v1)

176253

In [22]:
org_processing_df_v1.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,acquired,2005-06-01,2008-05-19,NaT,NaN
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,acquired,2004-10-11,2016-09-13,NaT,NaN
2,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,ipo,2004-02-04,2014-10-17,NaT,NaN


In [23]:
#merge companies with series a information
org_processing_df_v2 = pd.merge(org_processing_df_v1,
                                series_a_company_df,
                                on ='org_uuid',
                                how ='left')

In [24]:
org_processing_df_v2.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,acquired,2005-06-01,2008-05-19,NaT,NaN,2005-10-01,"42d516ee-bf29-387b-6946-3f2607aac85b,7ca12f7a-..."
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,acquired,2004-10-11,2016-09-13,NaT,NaN,2005-10-28,NaN
2,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,ipo,2004-02-04,2014-10-17,NaT,NaN,2005-05-01,b08efc27-da40-505a-6f9d-c9e14247bf36


In [25]:
len(org_processing_df_v2)

181779

In [26]:
#drop companies with no seed and series_a date
org_processing_df_v3 = org_processing_df_v2.dropna(subset=['seed_date','series_a_date'],
                                                  how='all')
len(org_processing_df_v3)

122513

In [27]:
org_processing_df_v3.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,acquired,2005-06-01,2008-05-19,NaT,NaN,2005-10-01,"42d516ee-bf29-387b-6946-3f2607aac85b,7ca12f7a-..."
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,acquired,2004-10-11,2016-09-13,NaT,NaN,2005-10-28,NaN
2,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,ipo,2004-02-04,2014-10-17,NaT,NaN,2005-05-01,b08efc27-da40-505a-6f9d-c9e14247bf36


In [28]:
#sort by date and drop duplicated rounds
org_processing_df_v3.sort_values(by=['seed_date','series_a_date'],inplace=True)
org_processing_df_v4 = org_processing_df_v3.drop_duplicates(subset=['org_uuid'],keep='first')

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
org_processing_df_v3[org_processing_df_v3.duplicated()]

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a
58425,1ee3a85c-de81-7b78-db37-e10d60a685fb,Fincore Ltd,operating,2006-11-01,2007-01-01,2007-01-01,NaN,NaT,NaN
3647,5b32cdb7-9530-a304-8f46-6afa1daf7ec4,Heyzap,acquired,2009-01-01,2012-12-01,2009-01-01,NaN,2010-06-23,b5d0d7dd-cfc2-4c56-9ed4-847db73f8cee
2892,dc690e72-7a3d-804f-f00a-8569587988d9,Formspring,closed,2009-11-25,2011-01-11,2010-01-15,NaN,2010-03-18,NaN
8424,195d6c58-87eb-174e-4e75-613a4043c07f,United Prototype,operating,2009-09-01,2011-02-08,2010-03-01,NaN,NaT,NaN
28522,817d5e6f-a75f-d9d8-2413-afaf9ffc3413,SoulPancake,acquired,2009-01-01,2010-04-01,2010-04-01,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
162972,8c26411c-9207-4f89-a4a3-267dcdeaa777,Lightnet,operating,2018-01-01,2020-01-11,NaT,NaN,2020-01-11,NaN
130728,0d37d3e5-1d27-4773-9ada-6ddb2185a2ba,Riparian Pharmaceuticals,operating,2012-01-01,2020-06-01,NaT,NaN,2020-06-01,NaN
98163,515185d7-29c4-8c67-72e3-c69588d20ba7,3AM Innovations,operating,2015-07-01,2020-06-25,NaT,NaN,2020-06-25,NaN
151696,d88eaf61-2268-4388-9c30-3fdd6e579c82,"Tabomedex Biosciences, Inc.",operating,2013-01-01,2020-08-03,NaT,NaN,2020-08-03,NaN


In [30]:
len(org_processing_df_v4)

93782

In [31]:
org_processing_df_v4.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a
39190,1f4f771f-e0f4-6b06-424b-adb5995951cc,Grand Junction,acquired,2013-10-01,2015-01-01,1993-06-01,NaN,NaT,NaN
74298,a006c611-beae-2533-574a-2fd052909433,Carrier Access Corporation,operating,2003-01-01,1997-06-01,1997-06-01,NaN,NaT,NaN
1300,5d2dafc0-e465-e861-06c9-62fb576e9c08,GasBuddy,acquired,2000-06-11,1999-12-01,1999-12-01,NaN,NaT,NaN


In [32]:
org_processing_df_v4['seed_to_series_a_years'] = ((org_processing_df_v4['series_a_date'] - org_processing_df_v4['seed_date']).dt.days/365).astype(float).round(1)
org_processing_df_v4.head(3)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years
39190,1f4f771f-e0f4-6b06-424b-adb5995951cc,Grand Junction,acquired,2013-10-01,2015-01-01,1993-06-01,NaN,NaT,NaN,NaN
74298,a006c611-beae-2533-574a-2fd052909433,Carrier Access Corporation,operating,2003-01-01,1997-06-01,1997-06-01,NaN,NaT,NaN,NaN
1300,5d2dafc0-e465-e861-06c9-62fb576e9c08,GasBuddy,acquired,2000-06-11,1999-12-01,1999-12-01,NaN,NaT,NaN,NaN


In [33]:
#select first fund date. Minimum of seed or series_a
org_processing_df_v4['first_fund_date'] = org_processing_df_v4[['seed_date','series_a_date']].min(axis=1)
org_processing_df_v4.head(3)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date
39190,1f4f771f-e0f4-6b06-424b-adb5995951cc,Grand Junction,acquired,2013-10-01,2015-01-01,1993-06-01,NaN,NaT,NaN,NaN,1993-06-01
74298,a006c611-beae-2533-574a-2fd052909433,Carrier Access Corporation,operating,2003-01-01,1997-06-01,1997-06-01,NaN,NaT,NaN,NaN,1997-06-01
1300,5d2dafc0-e465-e861-06c9-62fb576e9c08,GasBuddy,acquired,2000-06-11,1999-12-01,1999-12-01,NaN,NaT,NaN,NaN,1999-12-01


In [34]:
org_processing_df_v4['first_fund_date'] = pd.to_datetime(org_processing_df_v4['first_fund_date'])
org_processing_df_v4['founded_on'] = pd.to_datetime(org_processing_df_v4['founded_on'])

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
#calculate duration in years from founded to first fund
org_processing_df_v4['founded_to_first_fund_years'] = ((org_processing_df_v4['first_fund_date'] - org_processing_df_v4['founded_on']).dt.days/365).astype(float).round(1)
org_processing_df_v4.head(3)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years
39190,1f4f771f-e0f4-6b06-424b-adb5995951cc,Grand Junction,acquired,2013-10-01,2015-01-01,1993-06-01,NaN,NaT,NaN,NaN,1993-06-01,-20.3
74298,a006c611-beae-2533-574a-2fd052909433,Carrier Access Corporation,operating,2003-01-01,1997-06-01,1997-06-01,NaN,NaT,NaN,NaN,1997-06-01,-5.6
1300,5d2dafc0-e465-e861-06c9-62fb576e9c08,GasBuddy,acquired,2000-06-11,1999-12-01,1999-12-01,NaN,NaT,NaN,NaN,1999-12-01,-0.5


In [36]:
len(org_processing_df_v4)

93782

In [37]:
#filter company with more than 0 year and less than 4 year for first fund to occur
mask1 = org_processing_df_v4['founded_to_first_fund_years'] >=0
mask2 = org_processing_df_v4['founded_to_first_fund_years'] <=4
org_processing_df_v5 = org_processing_df_v4[mask1 & mask2] #option 1 - drop company does not receive first fund within 4 years
#org_processing_df_v5 = org_processing_df_v4[mask1] #option 2 - consider company does not receive first fund as failure, 0
len(org_processing_df_v5)

80048

In [38]:
org_processing_df_v5.isnull().sum(axis=0)

org_uuid                            0
name                                0
status                              0
founded_on                          0
last_funding_on                     0
seed_date                       15455
lead_investor_uuids_seed        63946
series_a_date                   53873
lead_investor_uuids_series_a    65091
seed_to_series_a_years          69328
first_fund_date                     0
founded_to_first_fund_years         0
dtype: int64

In [39]:
#add series_b date
org_processing_df_v6 = pd.merge(org_processing_df_v5,
                             investment_type.get_group('series_b')[['org_uuid','announced_on']],
                            on='org_uuid',
                            how = 'left')
#org_processing_df_v6.dropna(subset=['announced_on'],inplace=True)
org_processing_df_v6.rename(columns={'announced_on':'series_b_date'},inplace=True)
len(org_processing_df_v6)

81755

In [40]:
org_processing_df_v6.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date
0,61184ecb-b507-cba5-feb5-ad3226367aa2,eMerges.com,operating,2000-02-14,2000-02-14,2000-02-14,NaN,NaT,NaN,NaN,2000-02-14,0.0,NaT
1,50706f8c-551e-6d3b-e3ca-345a0ab6f8df,e-LogiT,operating,2000-02-14,2001-01-01,2000-02-14,NaN,NaT,NaN,NaN,2000-02-14,0.0,NaT
2,f9b46711-601c-0d15-cc58-20f838b51f92,iMusica,operating,2000-02-11,2008-01-01,2000-03-01,NaN,2004-04-01,NaN,4.1,2000-03-01,0.1,NaT


In [41]:
#add series c date
org_processing_df_v7 = pd.merge(org_processing_df_v6,
                                investment_type.get_group('series_c')[['org_uuid','announced_on']],
                                on='org_uuid',
                                how = 'left')
org_processing_df_v7.rename(columns={'announced_on':'series_c_date'},inplace=True)
len(org_processing_df_v7)

82632

In [42]:
org_processing_df_v7.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date
0,61184ecb-b507-cba5-feb5-ad3226367aa2,eMerges.com,operating,2000-02-14,2000-02-14,2000-02-14,NaN,NaT,NaN,NaN,2000-02-14,0.0,NaT,NaT
1,50706f8c-551e-6d3b-e3ca-345a0ab6f8df,e-LogiT,operating,2000-02-14,2001-01-01,2000-02-14,NaN,NaT,NaN,NaN,2000-02-14,0.0,NaT,NaT
2,f9b46711-601c-0d15-cc58-20f838b51f92,iMusica,operating,2000-02-11,2008-01-01,2000-03-01,NaN,2004-04-01,NaN,4.1,2000-03-01,0.1,NaT,NaT


In [43]:
#drop duplicate company
org_processing_df_v7.sort_values(by=['series_b_date','series_c_date'],axis=0,
                                 inplace=True,
                                 ignore_index=True)
org_processing_df_v8 = org_processing_df_v7.drop_duplicates(subset=['org_uuid'])
len(org_processing_df_v8)

80048

In [44]:
org_processing_df_v9 = pd.merge(org_processing_df_v8,
                                org_ori_df[['num_funding_rounds']],
                                on='org_uuid',
                                how='left')
org_processing_df_v9.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds
0,30b1084f-a57f-9dcd-d8b7-6985e8c76c14,Navic Networks,acquired,2000-01-03,2001-02-26,NaT,NaN,2000-02-14,ca1615fa-eef2-9e5d-f7ee-1c7195470a88,NaN,2000-02-14,0.1,2000-06-07,2001-02-26,3.0
1,2c640b60-6594-25ca-7d04-6d0e5ed4964d,Riot Entertainment,closed,2000-02-01,2000-10-13,NaT,NaN,2000-02-01,NaN,NaN,2000-02-01,0.0,2000-10-13,NaT,2.0
2,f7e7a793-3c80-02ed-eae7-1d095e32861a,Performance Logic,operating,2000-04-15,2000-12-21,NaT,NaN,2000-05-08,NaN,NaN,2000-05-08,0.1,2000-12-21,NaT,2.0


In [45]:
org_processing_df_v9['num_funding_rounds'].value_counts()

1.0     35410
2.0     18505
3.0     10636
4.0      6345
5.0      3783
6.0      2214
7.0      1281
8.0       768
9.0       441
10.0      219
11.0      160
12.0       93
13.0       60
14.0       38
15.0       22
16.0       18
17.0       14
18.0       11
19.0        6
20.0        4
22.0        4
23.0        4
21.0        3
25.0        2
31.0        2
29.0        1
24.0        1
35.0        1
30.0        1
27.0        1
Name: num_funding_rounds, dtype: int64

In [46]:
org_processing_df_v9.isnull().sum(axis=0)

org_uuid                            0
name                                0
status                              0
founded_on                          0
last_funding_on                     0
seed_date                       15455
lead_investor_uuids_seed        63946
series_a_date                   53873
lead_investor_uuids_series_a    65091
seed_to_series_a_years          69328
first_fund_date                     0
founded_to_first_fund_years         0
series_b_date                   68865
series_c_date                   75090
num_funding_rounds                  0
dtype: int64

In [47]:
#create dataframe for acquired companies
org_acquired_df = org_processing_df_v9[org_processing_df_v9['status'] == 'acquired']
len(org_acquired_df)

7863

In [48]:
org_acquired_df.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds
0,30b1084f-a57f-9dcd-d8b7-6985e8c76c14,Navic Networks,acquired,2000-01-03,2001-02-26,NaT,NaN,2000-02-14,ca1615fa-eef2-9e5d-f7ee-1c7195470a88,NaN,2000-02-14,0.1,2000-06-07,2001-02-26,3.0
5,6fb9ae50-a96d-13da-3cf9-acca7198674a,CRF Health,acquired,2000-09-01,2018-01-01,NaT,NaN,2000-10-25,1316efdc-ffea-ee63-873c-a122c4c4594f,NaN,2000-10-25,0.1,2001-02-15,2002-12-18,8.0
6,fa6d7097-5a28-e3ca-4cdc-c90a084738e7,Captus Networks,acquired,2000-04-01,2003-10-02,NaT,NaN,2000-12-01,NaN,NaN,2000-12-01,0.7,2001-05-02,2002-02-22,5.0


In [49]:
#merge with acquired_on date from original data frame
org_acquired_df_v1 = pd.merge(org_acquired_df,
                             org_ori_df['acquired_on'],
                             on='org_uuid',
                             how='left')
len(org_acquired_df_v1)

7863

In [50]:
org_acquired_df_v1.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on
0,30b1084f-a57f-9dcd-d8b7-6985e8c76c14,Navic Networks,acquired,2000-01-03,2001-02-26,NaT,NaN,2000-02-14,ca1615fa-eef2-9e5d-f7ee-1c7195470a88,NaN,2000-02-14,0.1,2000-06-07,2001-02-26,3.0,2008-06-18
1,6fb9ae50-a96d-13da-3cf9-acca7198674a,CRF Health,acquired,2000-09-01,2018-01-01,NaT,NaN,2000-10-25,1316efdc-ffea-ee63-873c-a122c4c4594f,NaN,2000-10-25,0.1,2001-02-15,2002-12-18,8.0,2013-01-01
2,fa6d7097-5a28-e3ca-4cdc-c90a084738e7,Captus Networks,acquired,2000-04-01,2003-10-02,NaT,NaN,2000-12-01,NaN,NaN,2000-12-01,0.7,2001-05-02,2002-02-22,5.0,2007-01-30


In [51]:
#drop company with no acquisition date
org_acquired_df_v2 = org_acquired_df_v1 #.dropna(subset=['acquired_on'])
len(org_acquired_df_v2)

7863

In [52]:
#calculate the duration from founded to acquisition and drop inconsistent data
org_acquired_df_v2['first_fund_to_acquired_years'] = ((org_acquired_df_v2['acquired_on'] - org_acquired_df_v2['first_fund_date']).dt.days/365).astype(float).round(decimals=1)

#drop company with acquire date before founded date
mask = org_acquired_df_v2['first_fund_to_acquired_years'] > 0
org_acquired_df_v3 = org_acquired_df_v2[mask]
len(org_acquired_df_v3)

6690

In [53]:
#merge with main processing df
org_processing_df_v10 = pd.concat([org_acquired_df_v3,
                                  org_processing_df_v9[org_processing_df_v9['status'] != 'acquired']],
                                  axis ='index',
                                  ignore_index=True)
len(org_processing_df_v10)

78875

In [54]:
org_processing_df_v10.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on,first_fund_to_acquired_years
0,30b1084f-a57f-9dcd-d8b7-6985e8c76c14,Navic Networks,acquired,2000-01-03,2001-02-26,NaT,NaN,2000-02-14,ca1615fa-eef2-9e5d-f7ee-1c7195470a88,NaN,2000-02-14,0.1,2000-06-07,2001-02-26,3.0,2008-06-18,8.3
1,6fb9ae50-a96d-13da-3cf9-acca7198674a,CRF Health,acquired,2000-09-01,2018-01-01,NaT,NaN,2000-10-25,1316efdc-ffea-ee63-873c-a122c4c4594f,NaN,2000-10-25,0.1,2001-02-15,2002-12-18,8.0,2013-01-01,12.2
2,fa6d7097-5a28-e3ca-4cdc-c90a084738e7,Captus Networks,acquired,2000-04-01,2003-10-02,NaT,NaN,2000-12-01,NaN,NaN,2000-12-01,0.7,2001-05-02,2002-02-22,5.0,2007-01-30,6.2


In [55]:
#create df for ipoed companies
org_ipo_df = org_processing_df_v10[org_processing_df_v10['status'] == 'ipo']
len(org_ipo_df)

889

In [56]:
#merge with announced date from original data frame and drop companies with no went_public_on date
org_ipo_df_v1 = pd.merge(org_ipo_df,
                         org_ori_df['went_public_on'],
                         on='org_uuid',
                         how='left')
org_ipo_df_v2 = org_ipo_df_v1.dropna(subset=['went_public_on'])
len(org_ipo_df_v2)

889

In [57]:
org_ipo_df_v2.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on,first_fund_to_acquired_years,went_public_on
0,d21a6980-f3a2-90ab-2255-2ab443ecaf18,Sumavision,ipo,2000-03-01,2009-08-01,NaT,NaN,2001-04-01,NaN,NaN,2001-04-01,1.1,2001-10-01,2004-11-01,7.0,NaT,NaN,2010-05-07
1,51ee0def-f0ac-c663-d359-d94bf7c2b9bf,Voltari,ipo,2001-01-01,2007-02-27,NaT,NaN,2001-01-01,NaN,NaN,2001-01-01,0.0,2002-01-01,NaT,5.0,NaT,NaN,2010-06-25
2,ef205a8a-ce41-9b7f-6a62-f9bcfc58a8b3,Aspen Aerogels,ipo,2001-01-01,2016-06-01,NaT,NaN,2001-05-17,8c951b3b-042d-062d-59aa-8f7e86e5979e,NaN,2001-05-17,0.4,2002-06-20,2003-12-24,11.0,NaT,NaN,2014-06-13


In [58]:
#calculate duration from first fund to IPO date and drop companies with duration <0
org_ipo_df_v2['first_fund_to_ipo_years'] = ((org_ipo_df_v2['went_public_on'] - org_ipo_df_v2['first_fund_date']).dt.days/365).astype(float).round(decimals=1)

mask = org_ipo_df_v2['first_fund_to_ipo_years'] > 0
org_ipo_df_v3 = org_ipo_df_v2[mask]
len(org_ipo_df_v3)

859

In [59]:
org_ipo_df_v3.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years
0,d21a6980-f3a2-90ab-2255-2ab443ecaf18,Sumavision,ipo,2000-03-01,2009-08-01,NaT,NaN,2001-04-01,NaN,NaN,2001-04-01,1.1,2001-10-01,2004-11-01,7.0,NaT,NaN,2010-05-07,9.1
1,51ee0def-f0ac-c663-d359-d94bf7c2b9bf,Voltari,ipo,2001-01-01,2007-02-27,NaT,NaN,2001-01-01,NaN,NaN,2001-01-01,0.0,2002-01-01,NaT,5.0,NaT,NaN,2010-06-25,9.5
2,ef205a8a-ce41-9b7f-6a62-f9bcfc58a8b3,Aspen Aerogels,ipo,2001-01-01,2016-06-01,NaT,NaN,2001-05-17,8c951b3b-042d-062d-59aa-8f7e86e5979e,NaN,2001-05-17,0.4,2002-06-20,2003-12-24,11.0,NaT,NaN,2014-06-13,13.1


In [60]:
#concatenate with main data frame
org_processing_df_v11 = pd.concat([org_ipo_df_v3,
                                   org_processing_df_v10[org_processing_df_v10['status']!='ipo']],
                                  axis=0,
                                  ignore_index=True)
len(org_processing_df_v11)

78845

In [61]:
org_processing_df_v11.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years
0,d21a6980-f3a2-90ab-2255-2ab443ecaf18,Sumavision,ipo,2000-03-01,2009-08-01,NaT,NaN,2001-04-01,NaN,NaN,2001-04-01,1.1,2001-10-01,2004-11-01,7.0,NaT,NaN,2010-05-07,9.1
1,51ee0def-f0ac-c663-d359-d94bf7c2b9bf,Voltari,ipo,2001-01-01,2007-02-27,NaT,NaN,2001-01-01,NaN,NaN,2001-01-01,0.0,2002-01-01,NaT,5.0,NaT,NaN,2010-06-25,9.5
2,ef205a8a-ce41-9b7f-6a62-f9bcfc58a8b3,Aspen Aerogels,ipo,2001-01-01,2016-06-01,NaT,NaN,2001-05-17,8c951b3b-042d-062d-59aa-8f7e86e5979e,NaN,2001-05-17,0.4,2002-06-20,2003-12-24,11.0,NaT,NaN,2014-06-13,13.1


In [62]:
#create data frame for closed companies
org_closed_df = org_processing_df_v11[org_processing_df_v11['status'] == 'closed']
len(org_closed_df)

5773

In [63]:
#merge with close and acquire date
org_closed_df_v1 = pd.merge(org_closed_df,
                           org_ori_df[['closed_on','acquired_on']],
                           on='org_uuid',
                           how='left')
len(org_closed_df_v1)

5773

In [64]:
org_closed_df_v1.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,...,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on_x,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,closed_on,acquired_on_y
0,2c640b60-6594-25ca-7d04-6d0e5ed4964d,Riot Entertainment,closed,2000-02-01,2000-10-13,NaT,NaN,2000-02-01,NaN,NaN,...,0.0,2000-10-13,NaT,2.0,NaT,NaN,NaT,NaN,NaT,NaT
1,3a9f4866-24fa-e703-ebdd-337f8a0b2c67,Corrent Corp.,closed,2000-08-01,2002-02-14,NaT,NaN,2000-12-01,7a42b564-4bb6-5864-6cdb-a0100008f3b3,NaN,...,0.3,2001-01-01,2002-02-14,3.0,NaT,NaN,NaT,NaN,NaT,NaT
2,eb37f7c8-c036-a915-bb7c-63f445330f66,Ceyba,closed,2000-05-01,2001-05-31,NaT,NaN,2000-06-26,beadb218-e5fa-2686-bc95-4dfaa9acc2e8,NaN,...,0.2,2001-05-31,NaT,2.0,NaT,NaN,NaT,NaN,2003-08-05,NaT


In [65]:
#get acquired companies from companies label as close. The companies are closed after acquired
org_acquired_df_v4 = org_closed_df_v1[org_closed_df_v1['acquired_on_y'].notna()]
len(org_acquired_df_v4)

400

In [66]:
org_acquired_df_v4.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,...,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on_x,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,closed_on,acquired_on_y
9,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,closed,2000-05-01,2005-04-01,NaT,NaN,2000-12-01,NaN,NaN,...,0.6,2002-02-19,2003-05-01,6.0,NaT,NaN,NaT,NaN,NaT,2006-12-04
12,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,closed,2000-10-01,2002-06-12,NaT,NaN,2001-02-12,NaN,NaN,...,0.4,2002-06-12,NaT,2.0,NaT,NaN,NaT,NaN,2004-01-01,2004-01-21
20,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,closed,2000-10-01,2007-08-06,NaT,NaN,2002-02-01,b2349e73-9f9b-a507-557c-634e0c528505,NaN,...,1.3,2003-07-11,2006-06-13,5.0,NaT,NaN,NaT,NaN,NaT,2009-03-10


In [67]:
#change the status to acquired
org_acquired_df_v4['status'] = 'acquired'

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
org_acquired_df_v4.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,...,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on_x,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,closed_on,acquired_on_y
9,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,NaT,NaN,2000-12-01,NaN,NaN,...,0.6,2002-02-19,2003-05-01,6.0,NaT,NaN,NaT,NaN,NaT,2006-12-04
12,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,NaT,NaN,2001-02-12,NaN,NaN,...,0.4,2002-06-12,NaT,2.0,NaT,NaN,NaT,NaN,2004-01-01,2004-01-21
20,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,NaT,NaN,2002-02-01,b2349e73-9f9b-a507-557c-634e0c528505,NaN,...,1.3,2003-07-11,2006-06-13,5.0,NaT,NaN,NaT,NaN,NaT,2009-03-10


In [69]:
#calculate the duration from founded to acquisition and drop inconsistent data
org_acquired_df_v4['first_fund_to_acquired_years'] = ((org_acquired_df_v4['acquired_on_y'] - org_acquired_df_v4['first_fund_date']).dt.days/365).astype(float).round(1)

#drop company with acquire date before founded date
mask = org_acquired_df_v4['first_fund_to_acquired_years'] > 0
org_acquired_df_v5 = org_acquired_df_v4[mask]
len(org_acquired_df_v5)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


395

In [70]:
org_acquired_df_v5.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,...,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on_x,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,closed_on,acquired_on_y
9,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,NaT,NaN,2000-12-01,NaN,NaN,...,0.6,2002-02-19,2003-05-01,6.0,NaT,6.0,NaT,NaN,NaT,2006-12-04
12,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,NaT,NaN,2001-02-12,NaN,NaN,...,0.4,2002-06-12,NaT,2.0,NaT,2.9,NaT,NaN,2004-01-01,2004-01-21
20,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,NaT,NaN,2002-02-01,b2349e73-9f9b-a507-557c-634e0c528505,NaN,...,1.3,2003-07-11,2006-06-13,5.0,NaT,7.1,NaT,NaN,NaT,2009-03-10


In [71]:
#modified column to make it consistent with main processing file
org_acquired_df_v5.drop(columns=['acquired_on_x','closed_on'],inplace=True)
org_acquired_df_v5.rename(columns={'acquired_on_y':'acquired_on'},inplace=True)
org_acquired_df_v5.head(3)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,acquired_on
9,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,NaT,NaN,2000-12-01,NaN,NaN,2000-12-01,0.6,2002-02-19,2003-05-01,6.0,6.0,NaT,NaN,2006-12-04
12,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,NaT,NaN,2001-02-12,NaN,NaN,2001-02-12,0.4,2002-06-12,NaT,2.0,2.9,NaT,NaN,2004-01-21
20,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,NaT,NaN,2002-02-01,b2349e73-9f9b-a507-557c-634e0c528505,NaN,2002-02-01,1.3,2003-07-11,2006-06-13,5.0,7.1,NaT,NaN,2009-03-10


In [72]:
#merge with main processing df
org_processing_df_v12 = pd.concat([org_acquired_df_v5,
                                  org_processing_df_v11[org_processing_df_v6['status'] != 'closed']],
                                 axis ='index',
                                 ignore_index=True)
len(org_processing_df_v12)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


73424

In [73]:
org_processing_df_v12.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,acquired_on
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,NaT,NaN,2000-12-01,NaN,NaN,2000-12-01,0.6,2002-02-19,2003-05-01,6.0,6.0,NaT,NaN,2006-12-04
1,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,NaT,NaN,2001-02-12,NaN,NaN,2001-02-12,0.4,2002-06-12,NaT,2.0,2.9,NaT,NaN,2004-01-21
2,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,NaT,NaN,2002-02-01,b2349e73-9f9b-a507-557c-634e0c528505,NaN,2002-02-01,1.3,2003-07-11,2006-06-13,5.0,7.1,NaT,NaN,2009-03-10


In [74]:
#get companies which are closed and not acquired
org_closed_df_v2 = org_closed_df_v1[org_closed_df_v1['acquired_on_y'].isna()]
len(org_closed_df_v2)

5373

In [75]:
org_closed_df_v2.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,...,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,acquired_on_x,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,closed_on,acquired_on_y
0,2c640b60-6594-25ca-7d04-6d0e5ed4964d,Riot Entertainment,closed,2000-02-01,2000-10-13,NaT,NaN,2000-02-01,NaN,NaN,...,0.0,2000-10-13,NaT,2.0,NaT,NaN,NaT,NaN,NaT,NaT
1,3a9f4866-24fa-e703-ebdd-337f8a0b2c67,Corrent Corp.,closed,2000-08-01,2002-02-14,NaT,NaN,2000-12-01,7a42b564-4bb6-5864-6cdb-a0100008f3b3,NaN,...,0.3,2001-01-01,2002-02-14,3.0,NaT,NaN,NaT,NaN,NaT,NaT
2,eb37f7c8-c036-a915-bb7c-63f445330f66,Ceyba,closed,2000-05-01,2001-05-31,NaT,NaN,2000-06-26,beadb218-e5fa-2686-bc95-4dfaa9acc2e8,NaN,...,0.2,2001-05-31,NaT,2.0,NaT,NaN,NaT,NaN,2003-08-05,NaT


In [76]:
#drop companies with no close date (optional)
org_closed_df_v3 = org_closed_df_v2.dropna(subset=['closed_on'])
len(org_closed_df_v3)

3556

In [77]:
#calculate duration from first fund to close and drop companies with inconsistent data
org_closed_df_v3['first_fund_to_close_years'] = ((org_closed_df_v3['closed_on'] - org_closed_df_v3['first_fund_date']).dt.days/365).astype(float).round(1)

mask = org_closed_df_v3['first_fund_to_close_years'] > 0
org_closed_df_v4 = org_closed_df_v3[mask]
len(org_closed_df_v4)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


3491

In [78]:
org_closed_df_v4.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,...,series_b_date,series_c_date,num_funding_rounds,acquired_on_x,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,closed_on,acquired_on_y,first_fund_to_close_years
2,eb37f7c8-c036-a915-bb7c-63f445330f66,Ceyba,closed,2000-05-01,2001-05-31,NaT,NaN,2000-06-26,beadb218-e5fa-2686-bc95-4dfaa9acc2e8,NaN,...,2001-05-31,NaT,2.0,NaT,NaN,NaT,NaN,2003-08-05,NaT,3.1
3,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,IceFyre Semiconductor,closed,2001-01-01,2004-07-13,NaT,NaN,2001-03-12,NaN,NaN,...,2001-08-08,2003-02-18,4.0,NaT,NaN,NaT,NaN,2005-05-01,NaT,4.1
4,de462c42-b0a5-c5af-9637-ec426b4e991f,ORMvision,closed,2000-12-01,2001-11-06,NaT,NaN,2001-05-01,NaN,NaN,...,2001-11-06,NaT,2.0,NaT,NaN,NaT,NaN,2007-01-01,NaT,5.7


In [79]:
#modified column to be consistent with main processing
org_closed_df_v4.drop(columns=['acquired_on_x','acquired_on_y'],inplace=True)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [80]:
org_processing_df_v12[org_processing_df_v12['status'] != 'closed'].head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,first_fund_date,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,acquired_on
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,NaT,NaN,2000-12-01,NaN,NaN,2000-12-01,0.6,2002-02-19,2003-05-01,6.0,6.0,NaT,NaN,2006-12-04
1,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,NaT,NaN,2001-02-12,NaN,NaN,2001-02-12,0.4,2002-06-12,NaT,2.0,2.9,NaT,NaN,2004-01-21
2,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,NaT,NaN,2002-02-01,b2349e73-9f9b-a507-557c-634e0c528505,NaN,2002-02-01,1.3,2003-07-11,2006-06-13,5.0,7.1,NaT,NaN,2009-03-10


In [81]:
#merge closed companies with main data frame
org_processing_df_v13 = pd.concat([org_processing_df_v12[org_processing_df_v12['status'] != 'closed'],
                                   org_closed_df_v4],
                                  axis=0,
                                  ignore_index=True)
len(org_processing_df_v13)

71590

In [82]:
org_processing_df_v13.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,...,founded_to_first_fund_years,series_b_date,series_c_date,num_funding_rounds,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,acquired_on,closed_on,first_fund_to_close_years
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,NaT,NaN,2000-12-01,NaN,NaN,...,0.6,2002-02-19,2003-05-01,6.0,6.0,NaT,NaN,2006-12-04,NaT,NaN
1,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,NaT,NaN,2001-02-12,NaN,NaN,...,0.4,2002-06-12,NaT,2.0,2.9,NaT,NaN,2004-01-21,NaT,NaN
2,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,NaT,NaN,2002-02-01,b2349e73-9f9b-a507-557c-634e0c528505,NaN,...,1.3,2003-07-11,2006-06-13,5.0,7.1,NaT,NaN,2009-03-10,NaT,NaN


In [83]:
#Calculate duration from first fund to next funding round
funding_rounds = ['series_a','series_b','series_c']

for invest_round in funding_rounds:
    new_column = str('first_fund_to_' + invest_round +'_years')
    exist_column = str (invest_round+'_date')
    org_processing_df_v13[new_column] = ((org_processing_df_v13[exist_column] - org_processing_df_v13['first_fund_date']).dt.days/365).astype(float).round(2)

org_processing_df_v13.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,...,num_funding_rounds,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,acquired_on,closed_on,first_fund_to_close_years,first_fund_to_series_a_years,first_fund_to_series_b_years,first_fund_to_series_c_years
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,NaT,NaN,2000-12-01,NaN,NaN,...,6.0,6.0,NaT,NaN,2006-12-04,NaT,NaN,0.0,1.22,2.41
1,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,NaT,NaN,2001-02-12,NaN,NaN,...,2.0,2.9,NaT,NaN,2004-01-21,NaT,NaN,0.0,1.33,NaN
2,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,NaT,NaN,2002-02-01,b2349e73-9f9b-a507-557c-634e0c528505,NaN,...,5.0,7.1,NaT,NaN,2009-03-10,NaT,NaN,0.0,1.44,4.36


In [84]:
end_date = pd.Timestamp(2020,12,31)
end_date

Timestamp('2020-12-31 00:00:00')

In [85]:
org_processing_df_v13['last_fund_to_end_date_years'] = ((end_date - org_processing_df_v13['last_funding_on']).dt.days/365).astype(float).round(2)
org_processing_df_v13.head(3)

,org_uuid,name,status,founded_on,last_funding_on,seed_date,lead_investor_uuids_seed,series_a_date,lead_investor_uuids_series_a,seed_to_series_a_years,...,first_fund_to_acquired_years,went_public_on,first_fund_to_ipo_years,acquired_on,closed_on,first_fund_to_close_years,first_fund_to_series_a_years,first_fund_to_series_b_years,first_fund_to_series_c_years,last_fund_to_end_date_years
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,NaT,NaN,2000-12-01,NaN,NaN,...,6.0,NaT,NaN,2006-12-04,NaT,NaN,0.0,1.22,2.41,15.76
1,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,NaT,NaN,2001-02-12,NaN,NaN,...,2.9,NaT,NaN,2004-01-21,NaT,NaN,0.0,1.33,NaN,18.57
2,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,NaT,NaN,2002-02-01,b2349e73-9f9b-a507-557c-634e0c528505,NaN,...,7.1,NaT,NaN,2009-03-10,NaT,NaN,0.0,1.44,4.36,13.41


In [86]:
org_processing_df_v13.columns

Index(['org_uuid', 'name', 'status', 'founded_on', 'last_funding_on',
       'seed_date', 'lead_investor_uuids_seed', 'series_a_date',
       'lead_investor_uuids_series_a', 'seed_to_series_a_years',
       'first_fund_date', 'founded_to_first_fund_years', 'series_b_date',
       'series_c_date', 'num_funding_rounds', 'first_fund_to_acquired_years',
       'went_public_on', 'first_fund_to_ipo_years', 'acquired_on', 'closed_on',
       'first_fund_to_close_years', 'first_fund_to_series_a_years',
       'first_fund_to_series_b_years', 'first_fund_to_series_c_years',
       'last_fund_to_end_date_years'],
      dtype='object')

In [87]:
#rearrange columns
org_processing_df_v14 = org_processing_df_v13[['org_uuid', 'name', 'status', 'founded_on','last_funding_on','num_funding_rounds',
                                              'seed_date','series_a_date', 'first_fund_date','series_b_date', 'series_c_date',
                                               'lead_investor_uuids_seed','lead_investor_uuids_series_a',
                                              'acquired_on','went_public_on','closed_on',
                                              'founded_to_first_fund_years','first_fund_to_series_a_years','first_fund_to_series_b_years','first_fund_to_series_c_years',
                                              'first_fund_to_acquired_years','first_fund_to_ipo_years','first_fund_to_close_years','last_fund_to_end_date_years']]

In [88]:
org_processing_df_v14.head(3)

,org_uuid,name,status,founded_on,last_funding_on,num_funding_rounds,seed_date,series_a_date,first_fund_date,series_b_date,...,went_public_on,closed_on,founded_to_first_fund_years,first_fund_to_series_a_years,first_fund_to_series_b_years,first_fund_to_series_c_years,first_fund_to_acquired_years,first_fund_to_ipo_years,first_fund_to_close_years,last_fund_to_end_date_years
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,6.0,NaT,2000-12-01,2000-12-01,2002-02-19,...,NaT,NaT,0.6,0.0,1.22,2.41,6.0,NaN,NaN,15.76
1,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,2.0,NaT,2001-02-12,2001-02-12,2002-06-12,...,NaT,NaT,0.4,0.0,1.33,NaN,2.9,NaN,NaN,18.57
2,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,5.0,NaT,2002-02-01,2002-02-01,2003-07-11,...,NaT,NaT,1.3,0.0,1.44,4.36,7.1,NaN,NaN,13.41


In [89]:
len(org_processing_df_v14)

71590

In [90]:
#select companies founded after 2000-01-01 and before 2013-12-31
mask1 = org_processing_df_v14['founded_on'] >= '2000-01-01'
mask2 = org_processing_df_v14['founded_on'] <= '2013-12-31'
org_processing_df_v15 = org_processing_df_v14[mask1 & mask2]

In [91]:
len(org_processing_df_v15)

28727

In [92]:
org_processing_df_v15['status'].value_counts()

operating    21277
acquired      4362
closed        2495
ipo            593
Name: status, dtype: int64

In [93]:
org_processing_df_v15.columns

Index(['org_uuid', 'name', 'status', 'founded_on', 'last_funding_on',
       'num_funding_rounds', 'seed_date', 'series_a_date', 'first_fund_date',
       'series_b_date', 'series_c_date', 'lead_investor_uuids_seed',
       'lead_investor_uuids_series_a', 'acquired_on', 'went_public_on',
       'closed_on', 'founded_to_first_fund_years',
       'first_fund_to_series_a_years', 'first_fund_to_series_b_years',
       'first_fund_to_series_c_years', 'first_fund_to_acquired_years',
       'first_fund_to_ipo_years', 'first_fund_to_close_years',
       'last_fund_to_end_date_years'],
      dtype='object')

In [94]:
#assign dependent variable, 1 for success and 0 for non-success
org_processing_df_v15.reset_index(drop=True,inplace=True)
outcome = list()
end = len(org_processing_df_v15)

for i in range(0,end):
    company = org_processing_df_v15.loc[i]
    #set success condition for operating company
    #company got funding series_a in within 3 years from first funding
    cond1 = company['first_fund_to_series_a_years'] <= 3 and company['first_fund_to_series_a_years'] > 0 #add condition more than 0 if first fund = series a
    #company got funding series_b 3 years from first funding
    cond2 = company['first_fund_to_series_b_years'] <= 3 
    #company got funding series_c within 3 years from first funding
    cond3 = company['first_fund_to_series_c_years'] <= 3
    #company last fund must be less than 3 years from end date (31/12/2020)
    cond4 = company['last_fund_to_end_date_years'] <=3
    
    #for company with closed status
    if company['status'] == 'closed':
        outcome.append(0)
    #for company with acquired and ipo status
    elif company ['status'] == 'acquired' or company['status'] == 'ipo':
        outcome.append (1)
    #for company with operating status
    elif company ['status'] == 'operating':
        #if company receive first fund ONLY after 4 years, consider as failure
        if company ['founded_to_first_fund_years'] > 4:
            outcome.append(0)
        else: #company receive first fund within 4 years and second fund within 3 years fron first fund and last fund must be less than 3 years from end date (31/12/2020)
            if (cond1 or cond2 or cond3) and cond4:
                outcome.append(1)
            else:
                outcome.append(0)
            
org_processing_df_v15['outcome'] = outcome

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [95]:
org_processing_df_v15['outcome'].value_counts()

0    22066
1     6661
Name: outcome, dtype: int64

In [96]:
success = (org_processing_df_v15[org_processing_df_v15.outcome == 1]).groupby('status').size().astype(int)
failure = (org_processing_df_v15[org_processing_df_v15.outcome == 0].groupby('status').size()).astype(int)

pd.concat ([success,failure], axis=1,keys = ['success','failure'])

,success,failure
status,,
acquired,4362.0,NaN
ipo,593.0,NaN
operating,1706.0,19571.0
closed,NaN,2495.0


In [97]:
org_processing_df_v15.groupby('status').size()

status
acquired      4362
closed        2495
ipo            593
operating    21277
dtype: int64

In [98]:
len(org_processing_df_v15)

28727

In [99]:
#save processed dataframe with outcome
org_processing_df_v15.to_csv(R"d:/msc-project/data/final/outcome_final_v1.csv",encoding='utf-8',index=False)

In [100]:
org_processing_df_v15.columns

Index(['org_uuid', 'name', 'status', 'founded_on', 'last_funding_on',
       'num_funding_rounds', 'seed_date', 'series_a_date', 'first_fund_date',
       'series_b_date', 'series_c_date', 'lead_investor_uuids_seed',
       'lead_investor_uuids_series_a', 'acquired_on', 'went_public_on',
       'closed_on', 'founded_to_first_fund_years',
       'first_fund_to_series_a_years', 'first_fund_to_series_b_years',
       'first_fund_to_series_c_years', 'first_fund_to_acquired_years',
       'first_fund_to_ipo_years', 'first_fund_to_close_years',
       'last_fund_to_end_date_years', 'outcome'],
      dtype='object')